In [1]:
import numpy as np
import open3d
import os
import tqdm
import copy
import time
import utils.fread as fread

from concurrent.futures import ThreadPoolExecutor

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [64]:
def compute_fpfh_features(pcd, voxel_size):
    radius_normal = voxel_size * 2
    radius_feature = voxel_size * 5
    
    pcd.estimate_normals(open3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))
    fpfh = open3d.pipelines.registration.compute_fpfh_feature(pcd, open3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd, fpfh

In [68]:
def execute_icp(source, target, trans_init, threshold, max_iteration):
    result = open3d.pipelines.registration.registration_icp(
        source, target, threshold, trans_init,
        open3d.pipelines.registration.TransformationEstimationPointToPlane(),
        open3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=max_iteration)
    )
    return result

In [69]:
def execute_ransac(source, target, source_feats, target_feats, n_ransac, threshold):
    result = open3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source, target, source_feats, target_feats, True,
        threshold,
        open3d.pipelines.registration.TransformationEstimationPointToPoint(False),
        n_ransac, [
            open3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
            open3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(threshold)
        ], open3d.pipelines.registration.RANSACConvergenceCriteria(4000000, 0.8))
    return result

In [107]:
def make_pcd(points):
    pcd = open3d.geometry.PointCloud()
    pcd.points = open3d.utility.Vector3dVector(points)
    return pcd


def get_limits(pcd):
    x_min, y_min, z_min = np.min(pcd, axis=0)
    x_max, y_max, z_max = np.max(pcd, axis=0)

    return x_min, x_max, y_min, y_max, z_min, z_max


def get_grid(pcd, cell_size):
    x_min, x_max, y_min, y_max, z_min, z_max = get_limits(pcd)
    y_val = np.mean([y_min, y_max])

    points = []
    x_n = int((x_max - x_min) // cell_size)
    z_n = int((z_max - z_min) // cell_size)
    for i in range(z_n):
        z0 = float(z_min + cell_size * (i + 1))
        for j in range(x_n):
            x0 = float(x_min + cell_size * (j + 1))
            points.append([x0, y_val, z0])

    return points


def filter_indices(points, p, cell_size):
    px_min = p[0] - cell_size
    px_max = p[0] + cell_size
    pz_min = p[2] - cell_size
    pz_max = p[2] + cell_size
    xf = np.logical_and(points[:, 0] > px_min, points[:, 0] < px_max)
    zf = np.logical_and(points[:, 2] > pz_min, points[:, 2] < pz_max)
    return np.logical_and(xf, zf)


def make_reg_features(feats):
    features = open3d.pipelines.registration.Feature()
    features.data = feats.T
    return features


def get_features(vertices, feats, n_random, voxel_size):
    n_keypts = len(vertices)
    indices = np.random.randint(0, n_keypts, int(n_keypts * n_random))
    
    keypts = make_pcd(vertices[indices])
    keypts.estimate_normals(open3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 2, max_nn=30))
    
    features = open3d.pipelines.registration.Feature()
    features.data = feats[indices].T
    return keypts, features


def get_cell_features(vertices, feats, p, cell_size, n_random, voxel_size):
    indices = filter_indices(vertices, p, cell_size)
    indices = np.where(indices)[0]
    
    if len(indices) < n_random:
        return None, None
    
    indices = np.random.choice(indices, int(len(indices) * n_random), replace=False)

    features = open3d.pipelines.registration.Feature()
    features.data = feats[indices].T

    keypts = make_pcd(vertices[indices])
    keypts.estimate_normals(open3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 2, max_nn=30))
    return keypts, features


def register_cell(source, target, source_feat, target_feat, n_ransac, threshold):
    return execute_ransac(source, target, source_feat, target_feat, n_ransac=n_ransac, threshold=threshold)


def global_registration(source, source_feat, global_pcd, global_feat, cell_size, voxel_size, n_random=0.5, refine_enabled=False):
    source, source_feat = get_features(source, source_feat, n_random, voxel_size)
    center_pts = get_grid(global_pcd, cell_size)
    
    targets = []
    target_feats = []
    
    delete_indices = []
    
    for i in range(len(center_pts)):
        target, target_feat = get_cell_features(global_pcd, global_feat, center_pts[i], cell_size, n_random, voxel_size)
        
        if not target or len(target.points) < 2000:
            delete_indices.append(i)
            continue
        
        targets.append(target)
        target_feats.append(target_feat)
        
    center_pts = np.delete(center_pts, delete_indices, axis=0)

    reg_result = None
    
    with ThreadPoolExecutor(max_workers=8) as executor:
        results = []
        for i in range(len(center_pts)):
            results.append(executor.submit(register_cell, source, targets[i], source_feat, target_feats[i], 3, 0.05))
            
        for i in range(len(center_pts)):
            result_ransac = results[i].result()
            
            if not result_ransac: continue
            
            if reg_result is None or (len(reg_result.correspondence_set) < len(result_ransac.correspondence_set) and reg_result.fitness < result_ransac.fitness):
                reg_result = result_ransac
    
    global_pcd = make_pcd(global_pcd)
    global_pcd.estimate_normals(open3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 2, max_nn=30))
    
    
    if refine_enabled and reg_result is not None:
        reg_result = execute_icp(source, global_pcd, threshold=0.05, trans_init=reg_result.transformation, max_iteration=200)
    
    return source, global_pcd, reg_result

In [108]:
# def read_fcgf_file(file, voxel_size):
#     data = np.load(file)
#     pcd = make_pcd(data.get("keypts"))

#     pcd.estimate_normals(open3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 2, max_nn=30))
    
#     features = make_reg_features(data.get("features"))
    
#     return pcd, features

def read_fcgf_file(file):
    data = np.load(file)
    return data.get("keypts"), data.get("features")

In [87]:
def view(source, target, T):
    p1 = copy.deepcopy(source)
    p2 = copy.deepcopy(target)
    
    p1.paint_uniform_color([1, 0.706, 0])
    p2.paint_uniform_color([0, 0.651, 0.929])
    
    p1.transform(T)
    
    open3d.visualization.draw_geometries([p1, p2])

Global registration with new open3d version

In [110]:
sequence_dir = "data/registration_sample/01"
voxel_size = 0.05

sequence_ts = fread.get_timstamps(sequence_dir, ".secondary.npz")

times = []

for t in tqdm.tqdm(range(0, len(sequence_ts), 20)):
    source, source_feat = read_fcgf_file(os.path.join(sequence_dir, f"{sequence_ts[t]}.secondary.npz"))
    target, target_feat = read_fcgf_file(os.path.join(sequence_dir, f"{sequence_ts[t]}.global.npz"))

    start_t = time.time()
    source, target, reg_result = global_registration(source, source_feat, target, target_feat, cell_size=3, voxel_size=voxel_size, n_random=0.7, refine_enabled=True)
    end_t = time.time()

    times.append(end_t - start_t)

    view(source, target, reg_result.transformation)

print(np.mean(times))

 32%|███▏      | 6/19 [00:18<00:40,  3.15s/it]


KeyboardInterrupt: 

In [55]:
sequence_dir = "data/registration_sample/01"

sequence_ts = fread.get_timstamps(sequence_dir, ".secondary.pcd")

voxel_size = 0.025

In [58]:
pcd = open3d.io.read_point_cloud(os.path.join(sequence_dir, f"{sequence_ts[65]}.secondary.pcd"))
pcd = pcd.voxel_down_sample(voxel_size)

open3d.visualization.draw_geometries([pcd])

pts = np.asarray(pcd.points)

np.save("data/demo/target.npy", pts)

In [54]:
pcd = open3d.io.read_point_cloud("../calibration/env.pcd")
pcd = pcd.voxel_down_sample(voxel_size)

open3d.visualization.draw_geometries([pcd])

# pts = np.asarray(pcd.points)

# np.save("../calibration/env.npy", pts)

In [23]:
local_pcds = []
local_fpfh = []

for t in tqdm.tqdm(sequence_ts):
    pcd = open3d.io.read_point_cloud(os.path.join(sequence_dir, f"{t}.secondary.pcd"))
    pcd = pcd.voxel_down_sample(voxel_size)
    pcd, fpfh = compute_fpfh_features(pcd, voxel_size)

    local_pcds.append(pcd)
    local_fpfh.append(fpfh)

# for t in range(1, len(sequence_ts)):
#     source = open3d.io.read_point_cloud(os.path.join(sequence_dir, f"{sequence_ts[t]}.secondary.pcd"))
#     target = open3d.io.read_point_cloud(os.path.join(sequence_dir, f"{sequence_ts[t - 1]}.secondary.pcd"))

#     source = source.voxel_down_sample(voxel_size)
#     target = target.voxel_down_sample(voxel_size)

#     source, source_fpfh = compute_fpfh_features(source, voxel_size)
#     target, target_fpfh = compute_fpfh_features(target, voxel_size)

#     source.paint_uniform_color([1, 0.706, 0])
#     target.paint_uniform_color([0, 0.651, 0.929])

#     open3d.visualization.draw_geometries([source, target])

#     reg_result = execute_ransac(source, target, source_fpfh, target_fpfh, voxel_size)
#     reg_result = execute_icp(source, target, reg_result.transformation, voxel_size)

#     source.transform(reg_result.transformation)

#     open3d.visualization.draw_geometries([source, target])

#     break

100%|██████████| 369/369 [00:15<00:00, 23.30it/s]


In [25]:
transformations = []

for t in tqdm.trange(len(sequence_ts) - 1):
    source = copy.deepcopy(local_pcds[t + 1])
    target = copy.deepcopy(local_pcds[t])

    source_fpfh = local_fpfh[t + 1]
    target_fpfh = local_fpfh[t]

    reg_result = execute_ransac(source, target, source_fpfh, target_fpfh, voxel_size)
    reg_result = execute_icp(source, target, reg_result.transformation, voxel_size)

    transformations.append(reg_result.transformation)

100%|██████████| 368/368 [00:56<00:00,  6.54it/s]


In [27]:
local_t = [np.identity(4)]

for t in tqdm.trange(len(sequence_ts) - 1):
    local_t.append(np.dot(local_t[t], transformations[t]))

100%|██████████| 368/368 [00:00<00:00, 497423.10it/s]


In [31]:
global_pcd = []

for t in range(len(local_t)):
    pcd = copy.deepcopy(local_pcds[t])
    pcd.transform(local_t[t])

    global_pcd.append(pcd)

In [112]:
import requests

In [141]:
source = open3d.io.read_point_cloud(os.path.join(sequence_dir, f"{sequence_ts[40]}.secondary.pcd"))
target = open3d.io.read_point_cloud(os.path.join(sequence_dir, f"{sequence_ts[40]}.global.pcd"))

source_pts = np.asarray(source.points).astype(np.float16).tolist()
target_pts = np.asarray(target.points).astype(np.float16).tolist()

In [142]:
# Define new data to create
new_data = {
    "source": source_pts,
    "target": target_pts
}

# The API endpoint to communicate with
url_post = "http://127.0.0.1:5557/register"

# A POST request to tthe API
response = requests.get(url_post, json=new_data)

# Print the response
post_response_json = response.json()
print(post_response_json)

[[0.0029239654541015625, 0.1502685546875, -0.98876953125, 2.0625], [-0.04840087890625, -0.9873046875, -0.1502685546875, 1.5087890625], [-0.9990234375, 0.048309326171875, 0.004390716552734375, 2.03515625], [0.0, 0.0, 0.0, 1.0]]


In [143]:
view(source, target, post_response_json)